# SQL Server Database Connection Setup

This notebook helps you:
1. Test connection to your SQL Server database
2. Explore available tables and views
3. Examine table schemas and sample data
4. Set up the foundation for ML model development

In [1]:
# Import required libraries
import sys
import os

# Add src to path for imports
sys.path.append('../src')

import pandas as pd
import numpy as np
from database.connection import SQLServerConnection, get_available_drivers
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Check Available ODBC Drivers

In [2]:
# Check available SQL Server ODBC drivers
available_drivers = get_available_drivers()
print("Available SQL Server ODBC drivers:")
for driver in available_drivers:
    print(f"  - {driver}")

if not available_drivers:
    print("No SQL Server ODBC drivers found!")
    print("Please install the Microsoft ODBC Driver for SQL Server.")
    print("Download from: https://docs.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server")

Available SQL Server ODBC drivers:
  - SQL Server
  - SQL Server Native Client 11.0
  - ODBC Driver 13 for SQL Server
  - SQL Server Native Client RDA 11.0
  - ODBC Driver 17 for SQL Server


## 2. Configure Database Connection

Make sure you've created a `.env` file with your database credentials.
Copy from `.env.example` and update with your actual values.

In [3]:
# Create database connection
# This will use credentials from your .env file
db = SQLServerConnection()

# Alternative: specify connection details directly (not recommended for production)
# db = SQLServerConnection(
#     server='your-server-name',
#     database='your-database-name',
#     username='your-username',
#     password='your-password'
# )

print("Database connection object created.")

Database connection object created.


## 3. Test Database Connection

In [4]:
# Test the connection
connection_success = db.test_connection()

if connection_success:
    print("✅ Database connection successful!")
    print(f"Connected to server: {db.server}")
    print(f"Database: {db.database}")
else:
    print("❌ Database connection failed!")
    print("Please check your connection settings in the .env file.")

✅ Database connection successful!
Connected to server: localhost\MSSQLSERVER01
Database: stockdata_db


## 4. Explore Database Structure

In [5]:
# Get list of all tables
if connection_success:
    tables_df = db.get_table_list()
    print(f"Found {len(tables_df)} tables:")
    display(tables_df)
else:
    print("Cannot retrieve tables - connection failed.")

2025-11-25 15:55:37,650 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2025-11-25 15:55:37,658 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-25 15:55:37,738 INFO sqlalchemy.engine.Engine SELECT schema_name()
2025-11-25 15:55:37,741 INFO sqlalchemy.engine.Engine [generated in 0.00371s] ()
2025-11-25 15:55:38,235 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2025-11-25 15:55:38,239 INFO sqlalchemy.engine.Engine [generated in 0.00400s] ()
2025-11-25 15:55:38,298 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2025-11-25 15:55:38,302 INFO sqlalchemy.engine.Engine [generated in 0.00449s] ()
2025-11-25 15:55:39,180 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-25 15:55:39,184 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLE

,schema_name,table_name,table_type
0,dbo,bad_tickers,BASE TABLE
1,dbo,nasdaq_100_hist_data,BASE TABLE
2,dbo,nasdaq_100_hist_data_bkp,BASE TABLE
3,dbo,nasdaq_top100,BASE TABLE
4,dbo,nse_500,BASE TABLE
5,dbo,nse_500_hist_data,BASE TABLE
6,dbo,nse_500_hist_data_bkp,BASE TABLE


In [6]:
# Get list of all views
if connection_success:
    views_df = db.get_view_list()
    print(f"Found {len(views_df)} views:")
    display(views_df)
else:
    print("Cannot retrieve views - connection failed.")

2025-11-25 15:55:47,075 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-25 15:55:47,077 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) OR [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max))) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2025-11-25 15:55:47,080 INFO sqlalchemy.engine.Engine [cached since 7.9s ago] ('BASE TABLE', 'VIEW', "\n        SELECT \n            TABLE_SCHEMA as schema_name,\n            TABLE_NAME as view_name,\n            TABLE_TYPE as table_type\n        FROM INFORMATION_SCHEMA.TABLES\n        WHERE TABLE_TYPE = 'VIEW'\n        ORDER BY TABLE_SCHEMA, TABLE_NAME\n        ", 'dbo')
2025-11-25 15:55:47,192 INFO sqlalchemy.engine.Engine 
        SELECT 
            TABLE_SCHEMA as schema_name,


,schema_name,view_name,table_type
0,dbo,nasdaq_100_atr,VIEW
1,dbo,nasdaq_100_atr_spikes,VIEW
2,dbo,nasdaq_100_bb_signals,VIEW
3,dbo,nasdaq_100_bollingerband,VIEW
4,dbo,nasdaq_100_ema_sma_view,VIEW
5,dbo,nasdaq_100_macd,VIEW
6,dbo,nasdaq_100_macd_signals,VIEW
7,dbo,nasdaq_100_RSI_calculation,VIEW
8,dbo,nasdaq_100_rsi_signals,VIEW
9,dbo,nasdaq_100_sma_signals,VIEW


## 5. Examine Table Schema and Sample Data

**Replace 'your_table_name' below with an actual table name from your database.**

In [9]:
# Examine a specific table (replace with your table name)
table_name = 'nasdaq_100_hist_data'  # UPDATE THIS
schema_name = 'dbo'  # UPDATE THIS IF NEEDED

if connection_success and table_name != 'your_table_name':
    try:
        # Get table schema
        schema_df = db.get_table_schema(table_name, schema_name)
        print(f"Schema for {schema_name}.{table_name}:")
        display(schema_df)
        
        # Get sample data
        sample_df = db.get_sample_data(table_name, schema_name, limit=10)
        print(f"\nSample data from {schema_name}.{table_name}:")
        display(sample_df)
        
    except Exception as e:
        print(f"Error examining table: {e}")
        print("Make sure the table name and schema are correct.")
else:
    print("Please update the table_name variable above with an actual table from your database.")

2025-11-25 15:56:52,943 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-25 15:56:52,948 INFO sqlalchemy.engine.Engine 
        SELECT 
            COLUMN_NAME as column_name,
            DATA_TYPE as data_type,
            IS_NULLABLE as is_nullable,
            COLUMN_DEFAULT as column_default,
            CHARACTER_MAXIMUM_LENGTH as max_length,
            NUMERIC_PRECISION as numeric_precision,
            NUMERIC_SCALE as numeric_scale
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_NAME = ? AND TABLE_SCHEMA = ?
        ORDER BY ORDINAL_POSITION
        
2025-11-25 15:56:52,951 INFO sqlalchemy.engine.Engine [generated in 0.00867s] ()
2025-11-25 15:56:52,955 INFO sqlalchemy.engine.Engine ROLLBACK


Query execution failed: (pyodbc.Error) ('07002', '[07002] [Microsoft][ODBC Driver 17 for SQL Server]COUNT field incorrect or syntax error (0) (SQLExecDirectW)')
[SQL: 
        SELECT 
            COLUMN_NAME as column_name,
            DATA_TYPE as data_type,
            IS_NULLABLE as is_nullable,
            COLUMN_DEFAULT as column_default,
            CHARACTER_MAXIMUM_LENGTH as max_length,
            NUMERIC_PRECISION as numeric_precision,
            NUMERIC_SCALE as numeric_scale
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_NAME = ? AND TABLE_SCHEMA = ?
        ORDER BY ORDINAL_POSITION
        ]
(Background on this error at: https://sqlalche.me/e/20/dbapi)


Error examining table: (pyodbc.Error) ('07002', '[07002] [Microsoft][ODBC Driver 17 for SQL Server]COUNT field incorrect or syntax error (0) (SQLExecDirectW)')
[SQL: 
        SELECT 
            COLUMN_NAME as column_name,
            DATA_TYPE as data_type,
            IS_NULLABLE as is_nullable,
            COLUMN_DEFAULT as column_default,
            CHARACTER_MAXIMUM_LENGTH as max_length,
            NUMERIC_PRECISION as numeric_precision,
            NUMERIC_SCALE as numeric_scale
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_NAME = ? AND TABLE_SCHEMA = ?
        ORDER BY ORDINAL_POSITION
        ]
(Background on this error at: https://sqlalche.me/e/20/dbapi)
Make sure the table name and schema are correct.


## 6. Custom Query Example

In [10]:
# Example: Execute a custom query
if connection_success:
    try:
        # Simple query to get database information
        query = """
        SELECT 
            DB_NAME() as database_name,
            GETDATE() as current_datetime,
            @@VERSION as sql_server_version
        """
        
        result_df = db.execute_query(query)
        print("Database Information:")
        display(result_df)
        
    except Exception as e:
        print(f"Error executing query: {e}")
else:
    print("Cannot execute query - connection failed.")

2025-11-25 15:57:03,690 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-25 15:57:03,692 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) OR [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max))) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2025-11-25 15:57:03,694 INFO sqlalchemy.engine.Engine [cached since 84.51s ago] ('BASE TABLE', 'VIEW', '\n        SELECT \n            DB_NAME() as database_name,\n            GETDATE() as current_datetime,\n            @@VERSION as sql_server_version\n        ', 'dbo')
2025-11-25 15:57:04,154 INFO sqlalchemy.engine.Engine 
        SELECT 
            DB_NAME() as database_name,
            GETDATE() as current_datetime,
            @@VERSION as sql_server_version
        
2025-11-2

,database_name,current_datetime,sql_server_version
0,stockdata_db,2025-11-25 15:57:04.170,Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 ...


## 7. Save Connection Info for Next Notebooks

In [11]:
# Save connection status for use in other notebooks
if connection_success:
    print("✅ Connection verified and ready for data exploration!")
    print("\nNext steps:")
    print("1. Open '02_data_exploration.ipynb' to analyze your data")
    print("2. Choose tables/views that contain data for your ML project")
    print("3. Start building your machine learning models")
    
    # Store some basic info
    connection_info = {
        'server': db.server,
        'database': db.database,
        'tables_count': len(tables_df),
        'views_count': len(views_df),
        'connection_verified': True
    }
    
    # You can pickle this for use in other notebooks if needed
    import pickle
    with open('../data/connection_info.pkl', 'wb') as f:
        pickle.dump(connection_info, f)
    
    print("\nConnection info saved to data/connection_info.pkl")
    
else:
    print("❌ Please fix the connection issues before proceeding.")
    print("\nTroubleshooting tips:")
    print("1. Verify your .env file has correct credentials")
    print("2. Check if SQL Server is running and accessible")
    print("3. Verify firewall settings allow connections")
    print("4. Ensure the ODBC driver is installed")

✅ Connection verified and ready for data exploration!

Next steps:
1. Open '02_data_exploration.ipynb' to analyze your data
2. Choose tables/views that contain data for your ML project
3. Start building your machine learning models

Connection info saved to data/connection_info.pkl
